# Offensive Language Identification in Social Media: Building a Model for Detecting and Categorising Offensive Tweets

This notebook combines the 3 models trained to categorise offensive tweets into 3 differnt levels: **A**, **B**, and **C**. **Level
A** is for identifying whether a post is **offensive** or
**not**. **Level B**, is for identifying whether a tweet contains an insult or threat to an individual, labelled as **"TIN"**, while **Level C** is for offence target identification. An instance is labelled as **"IND"** if the target of the offensive post is an individual, "GRP" if the target is a group, and **"OTH"** if the target does not belong to any of the previous two categories.

##Import Libraries

In [14]:
!python --version
!pip install transformers --quiet
!pip install pytorch-lightning --quiet

Python 3.9.16


In [15]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler

from transformers import BertTokenizerFast as BertTokenizer, BertModel, AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from transformers import logging
logging.set_verbosity_error()

import pytorch_lightning as pl
from torchmetrics.functional import auroc, accuracy
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

In [16]:
bert_model = bert = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

## Define Model Architectures

In [17]:
class OffensiveTweetTagger_taskA(pl.LightningModule):
  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.bert = bert_model
    # classifier: serve as a way to get the output of the BERT model and convert those into the num of classes which we want to predict
    self.classifier1 = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.dropOutLayer = nn.Dropout(p=0.1)
    self.relu = nn.ReLU()
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.classifier2 = nn.Linear(2, 1)
    # This criterion computes the cross entropy loss between input logits and target
    self.criterion = nn.BCELoss()
    
  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    # run linear layer ontop of the output
    output = self.classifier1(output.pooler_output)
    output = self.relu(output)
    output = self.dropOutLayer(output)
    output = self.classifier2(output)
    # apply sigmoid function
    output = torch.sigmoid(output)
    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']
    loss, outputs = self.forward(input_ids, attention_mask, labels)
    # output training loss
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']
    loss, outputs = self.forward(input_ids, attention_mask, labels)
    # output training loss
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss
  
  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']
    loss, outputs = self.forward(input_ids, attention_mask, labels)
    # output training loss
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  # configer optimisers and learning rate scheduler
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=5e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=self.n_warmup_steps,
        num_training_steps=self.n_training_steps
    )

    # return a list of optimisers and schedulers
    return dict(
        optimizer=optimizer,
        lr_scheduler=dict(
            scheduler=scheduler,
            interval='step'
        )
    )

In [18]:
class OffensiveTweetTagger_taskC(pl.LightningModule):
  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.bert = bert_model
    # classifier: serve as a way to get the output of the BERT model and convert those into the num of classes which we want to predict
    self.classifier1 = nn.Linear(self.bert.config.hidden_size, 16)
    self.dropOutLayer = nn.Dropout(p=0.1)
    self.relu = nn.ReLU()
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.classifier2 = nn.Linear(16, n_classes)
    # This criterion computes the cross entropy loss between input logits and target
    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    # run linear layer ontop of the output
    output = self.classifier1(output.pooler_output)
    output = self.relu(output)
    output = self.dropOutLayer(output)
    output = self.classifier2(output)
    # apply sigmoid function
    output = torch.sigmoid(output)
    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']
    loss, outputs = self.forward(input_ids, attention_mask, labels)
    # output training loss
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']
    loss, outputs = self.forward(input_ids, attention_mask, labels)
    # output training loss
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss
  
  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']
    loss, outputs = self.forward(input_ids, attention_mask, labels)
    # output training loss
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  # configer optimisers and learning rate scheduler
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=5e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=self.n_warmup_steps,
        num_training_steps=self.n_training_steps
    )

    # return a list of optimisers and schedulers
    return dict(
        optimizer=optimizer,
        lr_scheduler=dict(
            scheduler=scheduler,
            interval='step'
        )
    )

##Pre-Process Tweet

In [19]:
def PreProcessing(tweet: str, tokenizer: AutoTokenizer, max_token_len: int=128):
  encoding = tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,
        max_length=max_token_len,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt"
    )
  return dict(
        tweet=tweet,
        input_ids=encoding["input_ids"],
        attention_mask=encoding["attention_mask"]
        )   

##Load Trained Models

In [28]:
#Trained model for subtask_A
!gdown 104mHPrMOJ6ayMgzV0oiE46gOHtG-greh
#Trained model for subtask_B
!gdown 1-2dmyv3Zb9W-ZDSPjlTUE9Arg5xF2foh
#Trained model for subtask_C
!gdown 1--kGa5q49ggK4hNOeAHFHQP3NkYXP6kN


Downloading...
From: https://drive.google.com/uc?id=104mHPrMOJ6ayMgzV0oiE46gOHtG-greh
To: /content/subtask_a_best_model.ckpt
100% 438M/438M [00:09<00:00, 44.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-2dmyv3Zb9W-ZDSPjlTUE9Arg5xF2foh
To: /content/subtask_b_best_model.ckpt
100% 438M/438M [00:01<00:00, 219MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--kGa5q49ggK4hNOeAHFHQP3NkYXP6kN
To: /content/subtask_c_best_model.ckpt
100% 438M/438M [00:01<00:00, 260MB/s]


In [38]:
def ClassifyTweet(tweet):
  trained_model_A = OffensiveTweetTagger_taskA.load_from_checkpoint(
    "subtask_a_best_model.ckpt",
    n_classes=2
  )
  trained_model_A.eval()
  trained_model_A.freeze()

  trained_model_B = OffensiveTweetTagger_taskA.load_from_checkpoint(
    "subtask_b_best_model.ckpt",
    n_classes=2
  )
  trained_model_B.eval()
  trained_model_B.freeze()

  trained_model_C = OffensiveTweetTagger_taskC.load_from_checkpoint(
    "subtask_c_best_model.ckpt",
    n_classes=3
  )
  trained_model_C.eval()
  trained_model_C.freeze()

  model_A_input = PreProcessing(tweet, tokenizer, 80)
  model_B_input = PreProcessing(tweet, tokenizer, 50)
  model_C_input = PreProcessing(tweet, tokenizer, 60)

  trained_model_A.to('cpu')
  trained_model_B.to('cpu')
  trained_model_C.to('cpu')

  print("Tweet: ", tweet)

  # Calculate Predictions
  _, test_prediction_a = trained_model_A(model_A_input['input_ids'], model_A_input['attention_mask'])
  test_prediction_a = test_prediction_a.flatten().numpy()
  _, test_prediction_b = trained_model_B(model_B_input['input_ids'], model_B_input['attention_mask'])
  test_prediction_b = test_prediction_b.flatten().numpy()
  _, test_prediction_c = trained_model_C(model_C_input['input_ids'], model_C_input['attention_mask'])
  test_prediction_c = test_prediction_c.flatten().numpy()
  test_prediction_c[test_prediction_c.argmax()] = 1

  print("Subtask_A: ", end="")
  if(test_prediction_a[0] > 0.5):
    print("This tweet is OFFENSIVE")
  else:
    print("This tweet is NOT OFFENSIVE")
    return
  print("Subtask_B: ", end="")
  if(test_prediction_b[0] > 0.5):
    print("This tweet is a Targeted Insult")
  else:
    print("This tweet is Untargeted")
    return
  print("Subtask_C: ", end="")
  if (test_prediction_c[0]==1):
    print("This tweet is targeted at an INDividual")
  elif (test_prediction_c[1]==1):
    print("This tweet is targeted at OTH than an individual or group")
  else:
    print("This tweet is targeted at a Group (GRP)")

In [40]:
tweet = input("Enter a tweet to classify: ")
ClassifyTweet(tweet)

Enter a tweet to classify: #FortniteBattleRoyale #XboxShare @USER   Please ban this cheating scum.  he is literally invisible URL
Tweet:  #FortniteBattleRoyale #XboxShare @USER   Please ban this cheating scum.  he is literally invisible URL
Subtask_A: This tweet is OFFENSIVE
Subtask_B: This tweet is a Targeted Insult
Subtask_C: This tweet is targeted at a Group (GRP)
